# III. 실습: 다양한 페이로드 테스트 및 분석

목표
- 정상과 악성 페이로드에 대해 모델의 반응을 직접 확인
- 임계값(threshold) 조정으로 민감도/정밀도 균형 이해
- 오탐(False Positive)과 미탐(False Negative) 사례 확인


### 환경 설정

In [ ]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from utils.TH_BERT import setting_bert_model, tokenizer
bert_model_file = "./model/bert_classification.pth"  
max_len = 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = setting_bert_model(device, bert_model_file, output_numbers=12)
print("device:", device)


In [ ]:
def predict_texts(texts, tokenizer, model, max_length=max_len):
    model.eval()
    inputs = tokenizer(
        texts,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    token_type_ids = inputs["token_type_ids"]
    ids = torch.tensor(ids, dtype=torch.long)
    attention_mask = torch.tensor(mask, dtype=torch.long)
    token_type_ids =  torch.tensor(token_type_ids, dtype=torch.long)
    with torch.no_grad():
        outputs = model(ids, attention_mask, token_type_ids)
        probs = torch.softmax(outputs, dim=-1).detach().cpu().numpy()
    pred_ids = probs.argmax(axis=1)
    pred_scores = probs.max(axis=1)
    return pred_ids, pred_scores, probs

In [ ]:
CLASS_NAMES = [
    'Benign',
    'Banking_Emotet_Family',
    'Banking_Zeus_Family',
    'Banking_Other',
    'Infostealer',
    'RAT',
    'Exploit_Kit',
    'Malspam_Phishing',
    'Ransomware',
    'Recon_C2',
    'Other_Generic',
    'Cobalt_Strike'
]


### 입력 결과 파일 준비

In [ ]:
from utils import data_load
# data setting
data_base_dir = "./model_data/"
bert_file = "bert_inputs.txt"
MAXLEN = 512
nlp_data_label = data_load(base_dir=data_base_dir, bert_flie=bert_file, MAXLEN=MAXLEN)

### 3.1정상/악성 페이로드 입력
- 준비된 샘플(`validation_result.csv`) 입력  
- BERT 분류기의 출력 확인
- 결과를 `validation_result_{label_name}.csv`로 저장  

In [ ]:
labels_data = {}
nlp_datas = nlp_data_label[0]
nlp_labels = nlp_data_label[1]
for text, label in zip(nlp_datas, nlp_labels):
    temp_list = []
    if label in labels_data:
        temp_list = labels_data[label]
    else:
        temp_list = []
    temp_list.append(text)
    labels_data[label] = temp_list

In [ ]:
print(labels_data.get(0))

In [ ]:
import random
def get_class_data(data, label_num, k, seed=42):
    random.seed(seed)
    class_data = data.get(label_num)
    return random.sample(class_data, k)

In [ ]:
label_num = 10
label_name = CLASS_NAMES[label_num]
input_texts = get_class_data(labels_data, label_num, 10, 42)
pred_ids, pred_scores, probs = predict_texts(input_texts, tokenizer, bert_model)
result_df = pd.DataFrame(list(zip(
    input_texts,
    pred_ids,
    pred_scores,
    probs
)), columns=["text", "predicted_label", "predicted_score", "probabilities"])

display(result_df)
result_df.to_csv(f"validation_result_{label_name}.csv", index=False)

In [ ]:
label_num = 4
label_name = CLASS_NAMES[label_num]
input_texts = get_class_data(labels_data, label_num, 10, 42)
pred_ids, pred_scores, probs = predict_texts(input_texts, tokenizer, bert_model)
result_df = pd.DataFrame(list(zip(
    input_texts,
    pred_ids,
    pred_scores,
    probs
)), columns=["text", "predicted_label", "predicted_score", "probabilities"])

display(result_df)
result_df.to_csv(f"validation_result_{label_name}.csv", index=False)

### 예시 분석: 왜 이 요청은 악성(Malicious)으로 분류되었을까?

---

**🧾 입력 샘플:**
Detected Protocol: HTTP(S) Entropy: 7.97 Encryption Status: Possibly Encrypted Signature: 474554202f204854 (first 8 bytes) Length: 35180 bytes Readable ASCII Data: GET / HTTP/1.1Host: montanamanshops.comUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8Accept-Language: en-US,en;q=0.5Accept-Encoding: gzip, deflateReferer: https://www.google.com/Connection: keep-aliveUpgrade-Insecure-Reque

**🧠 모델 예측 결과:**

- **예측 클래스:** `Infostealer`
- **예측 확률:** `98.7%`

---

### 왜 악성일까? (모델이 반응한 특징)

| 항목 | 설명 |
|------|------|
| **Host 도메인** | `montanamanshops.com` — 정상 사이트로 보이지만, 실제 존재하지 않거나 공격자가 만든 C2 서버일 가능성 |
| **Payload 길이** | `35180 bytes` → 단순 GET 요청으로는 지나치게 김 |
| **엔트로피** | `7.97` → 암호화된 명령이나 인코딩된 공격 코드 포함 가능성 |
| **Referer** | `https://www.google.com` — 공격 우회 또는 리디렉션을 가장하려는 흔한 기법 |
| **User-Agent** | 최신 브라우저 UA를 위장하여 탐지 우회 시도 가능성 |
| **패턴 특징** | 일반적인 HTTP 구조를 유지하면서도 공격 의심되는 조합이 많음 |

---

### 결론

> "이 요청은 언뜻 보기엔 정상적인 웹 요청처럼 보이지만,  
비정상적인 도메인, 비정상적인 바이트 크기, 높은 엔트로피 등을 종합적으로 고려하면  
**악성 C2 통신 또는 정찰 목적의 요청일 가능성이 높습니다.**"

> "모델은 단순히 단어를 보고 판단하는 것이 아니라,  
이러한 패턴의 **은밀한 통신 방식**을 학습한 결과, Recon_C2로 판단한 것입니다."


### 3.2 임계값(Threshold) 조정 실습

모델은 각 샘플에 대해 악성일 확률(`prob`)을 출력합니다.  
우리는 이 확률이 일정 값(threshold)을 넘는 경우만 **악성으로 예측**하게 설정할 수 있습니다.

---

- **Threshold가 낮으면**: 모델은 많은 샘플을 악성으로 판단 → **Recall(재현율)** ↑, 하지만 **오탐(정밀도)** ↓  
- **Threshold가 높으면**: 모델은 더 확신 있는 경우에만 악성으로 판단 → **Precision(정밀도)** ↑, 하지만 **재현율(Recall)** ↓

---

**그래프를 통해 Precision, Recall, F1-score가 threshold에 따라 어떻게 변화하는지를 확인합니다.**

이를 통해 **업무 목적에 따라 민감도 중심 탐지 vs. 정밀도 중심 탐지** 설정을 조절할 수 있습니다.


### Threshold(임계값) 조정이 중요한 이유

모델은 각 데이터에 대해 "악성일 확률"을 출력합니다.  
하지만 **그 확률이 몇 % 이상이면 악성이라고 판단할지**는 우리가 정해야 합니다.  
이 기준값을 **threshold (임계값)** 이라고 합니다.

---

### 현실 비유: 공항 보안 검색대

공항에서 금속탐지기를 생각해봅시다.

| Threshold 설정 | 실제 동작 | 효과 |
|----------------|------------|------|
| 낮게 설정       | 동전, 벨트만 있어도 삐- | 민감하게 작동 → 악성은 잘 잡힘, 하지만 정상도 자주 걸림 |
| 높게 설정       | 총처럼 확실할 때만 삐-  | 신중하게 작동 → 정상은 잘 보호됨, 하지만 약한 악성은 놓칠 수도 있음 |

---

### 사이버 보안에서의 의미

| Threshold | 예시 판단 기준 | 설명 |
|-----------|----------------|------|
| 0.3 | 30% 확률만 넘어도 악성 | 많이 탐지됨 (Recall↑), 대신 오탐도 증가 가능 |
| 0.5 | 중립적인 기준점       | 많이 사용되는 일반적 기준 |
| 0.8 | 80% 확신이 있어야 악성 | 정밀하게 탐지 (Precision↑), 하지만 미탐 위험 있음 |

---

### 실무 적용 예시

| 상황 | 중요한 지표 |
|------|--------------|
| APT 위협 탐지 | Recall (한 건도 놓치지 않기) |
| 자동 차단 시스템 | Precision (정상은 절대 막지 않기) |

---

### 실습 목표

- `validation_result_all.csv` 파일에는 모델의 예측 결과가 포함되어 있습니다.  
- 다양한 Threshold 값을 적용하며, Precision / Recall / F1-score가 어떻게 바뀌는지 관찰합니다.
- 이 실습을 통해 **탐지 민감도 vs 정밀도 사이의 균형**을 직접 체감해볼 수 있습니다.


#### file load

In [ ]:
import pandas as pd

df = pd.read_csv("validation_result.csv")
print(df.head())
print(df.describe())

#### Threshold에 따른 에측값 재구성

In [ ]:
def apply_threshold(prob, threshold):
    return [1 if p >= threshold else 0 for p in prob]

#### 민감도(Recall), 정밀도(Precision) 계산

# Multiclass Classification에서 `average` 옵션 비교

멀티클래스 분류에서 `precision_score`, `recall_score`, `f1_score` 등을 계산할 때  
`average` 파라미터를 어떻게 설정하느냐에 따라 결과가 달라집니다.

---

##### 1. `None`
- 클래스별 점수를 **그대로 반환**
- 예: 클래스가 3개라면 → `[0.82, 0.75, 0.90]`
- **활용**: 각 클래스별 성능을 개별적으로 확인할 때  
  (어떤 클래스에서 성능이 낮은지 분석 가능)

---

##### 2. `micro`
- 모든 클래스를 합쳐서 **전체적인 관점**에서 평가
- 방식: 전체 TP, FP, FN을 합산한 후 계산
- 특징: 데이터 불균형에 강하지만, 소수 클래스의 성능은 묻힘
- **활용**: 전체적인 "모델의 전반적인 성능"을 확인하고 싶을 때

---

##### 3. `macro`
- 클래스별 점수를 계산한 뒤, **단순 평균**
- 모든 클래스에 **동일 가중치**
- 특징: 클래스 불균형에 민감 (소수 클래스 성능이 낮으면 전체도 낮아짐)
- **활용**: 모든 클래스를 **동등하게 중요시**할 때 적합

---

##### 4. `weighted`
- 클래스별 점수에 **클래스 샘플 수(가중치)**를 반영한 평균
- 데이터가 많은 클래스의 영향력이 큼
- **활용**: 데이터 분포를 고려한 **현실적인 성능 평가**에 적합

---

## 한눈에 비교

| 옵션       | 계산 방식                  | 장점                              | 단점                          |
|------------|---------------------------|-----------------------------------|-------------------------------|
| `None`     | 클래스별 점수 반환         | 세부 분석 가능                     | 종합적인 판단 어려움           |
| `micro`    | 전체 TP/FP/FN 합산 후 계산 | 전체적인 성능, 불균형에 강함       | 소수 클래스 성능 무시됨        |
| `macro`    | 클래스별 점수 단순 평균    | 모든 클래스 동등하게 평가 가능     | 데이터 불균형에 민감           |
| `weighted` | 클래스별 점수 × 샘플 비율  | 현실적인 성능 반영 (불균형 대응)   | 데이터가 많은 클래스에 치우침  |


### Threshold 조정 실험 (슬라이더 방식)

아래 슬라이더를 움직여 보면서,  
모델의 예측 임계값(threshold)에 따라 Precision / Recall / F1 Score가 어떻게 달라지는지 실시간으로 확인해보세요.


In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
import ipywidgets as widgets
from IPython.display import display

df = pd.read_csv("validation_result.csv")
for col in ["target", "output"]:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(-1).astype(int)
df["preds"] = pd.to_numeric(df["preds"], errors="coerce").fillna(0.0).astype(float)

classes = sorted(set(df["target"]) | set(df["output"]))
assert classes, "라벨이 비었습니다."

class_slider = widgets.SelectionSlider(
    options=[(f"Class {c}", c) for c in classes],
    value=classes[0], description="Class:", continuous_update=False,
)
conf_slider = widgets.FloatSlider(
    value=0.00, min=0.0, max=1.0, step=0.01,
    description="conf >=", continuous_update=False, readout_format=".2f",
)
avg_dropdown = widgets.Dropdown(
    options=[("micro", "micro"), ("macro", "macro"), ("weighted", "weighted")],
    value="macro", description="avg:",
)

report = widgets.HTML(value="")
clear_btn = widgets.Button(description="출력 지우기")
clear_btn.on_click(lambda _: setattr(report, "value", ""))

def update(_=None):
    conf = float(conf_slider.value)
    selected_class = class_slider.value
    avg = avg_dropdown.value

    dff = df.loc[df["preds"] >= conf]
    total_n, kept_n = len(df), len(dff)
    cov = (kept_n / total_n) if total_n else 0.0

    lines = [f"conf >= {conf:.2f}", f"coverage: {kept_n} / {total_n} ({cov:.2f})"]

    if kept_n:
        y_true = dff["target"].values
        y_pred = dff["output"].values

        prec = precision_score(y_true, y_pred, average=avg, zero_division=0)
        rec = recall_score(y_true, y_pred, average=avg, zero_division=0)
        f1 = f1_score(y_true, y_pred, average=avg, zero_division=0)
        lines += ["", f"overall (average={avg})",
                  f"Precision: {prec:.4f}",
                  f"Recall:    {rec:.4f}",
                  f"F1 Score:  {f1:.4f}"]

        y_true_bin = (y_true == selected_class).astype(int)
        y_pred_bin = (y_pred == selected_class).astype(int)
        p = precision_score(y_true_bin, y_pred_bin, zero_division=0)
        r = recall_score(y_true_bin, y_pred_bin, zero_division=0)
        f = f1_score(y_true_bin, y_pred_bin, zero_division=0)
        s = int((y_true == selected_class).sum())
        lines += ["", f"class {selected_class} (one-vs-rest)",
                  f"Precision: {p:.4f}",
                  f"Recall:    {r:.4f}",
                  f"F1 Score:  {f:.4f}",
                  f"Support:   {s}"]

    report.value = "<pre>" + "\n".join(lines) + "</pre>"

for w in (class_slider, conf_slider, avg_dropdown):
    w.unobserve_all()
class_slider.observe(update, names="value")
conf_slider.observe(update, names="value")
avg_dropdown.observe(update, names="value")

ui = widgets.VBox([widgets.HBox([class_slider, avg_dropdown]), conf_slider, clear_btn])
layout = widgets.VBox([ui, report])
display(layout)

update() 


### 결과 해석 가이드

- **Precision(정밀도)**:  
  → 모델이 악성이라고 한 것 중 실제로 진짜 악성인 비율입니다.  
  → 수치가 높으면 "거의 틀리지 않게 악성만 잘 골라냄"입니다.

- **Recall(재현율)**:  
  → 실제 악성 중에서 모델이 얼마나 많이 잡아냈는지를 뜻합니다.  
  → 수치가 높으면 "악성을 놓치지 않고 잘 탐지함"입니다.

- **F1 Score**:  
  → 정밀도와 재현율의 균형을 본 점수입니다.  
  → 두 성능이 고르게 좋을수록 F1이 높습니다.

---

### 해석 예시

- `Precision이 높고 Recall이 낮다`:  
  → "실수는 안 하지만 너무 조심스러워서 많은 악성은 놓침"

- `Recall이 높고 Precision이 낮다`:  
  → "악성은 다 잡지만 정상이 너무 많이 오탐됨"

- `F1이 가장 높을 때`:  
  → "정밀도와 재현율이 적절히 균형잡힌 상태"

---

### 현실 적용 예시

- **자동 차단 시스템**  
  → Precision을 높게 설정해야 오탐(정상 차단)을 막을 수 있어요.

- **위협 헌팅, 탐지 경고 시스템**  
  → Recall을 높여 놓치지 않는 것이 중요해요.

- **종합적인 시스템 평가**  
  → F1 Score를 기준으로 모델을 비교합니다.


In [ ]:
import pandas as pd

# 예측 결과 로딩
df = pd.read_csv("validation_result.csv")

inaccuracy = df[(df["target"] != df["output"])]
inaccuracy = inaccuracy.set_index('pid')

df_text = pd.DataFrame({
    "pid": nlp_data_label[2],
    "text": nlp_data_label[0]
}).set_index("pid")

inaccuracy_df = inaccuracy.join(df_text, how="left")

inaccuracy_df.to_csv("inaccuracy_samples.csv", index=True)
print(f"오탐 샘플 저장 완료: {len(inaccuracy_df)}개")


### 오탐 샘플 해설 (실제: Benign, 예측: Banking_Emotet_Family)

---

#### 샘플 1 – `pid=1572623`
- **예측 확률:** 80.3%
- **실제:** Benign
- **예측:** Banking_Emotet_Family

**요약 내용:**
```
Detected Protocol: HTTP(S)
Entropy: 5.50
Encryption Status: Possibly Encrypted
Signature: 474554202f204854 (GET / HT)
Length: 409 bytes

GET / HTTP/1.1
Host: columbusadvocate.com
User-Agent: Mozilla/5.0 ...
```


**해석:**
- `Host: columbusadvocate.com` → 생소한 뉴스/블로그 스타일 도메인
- `GET /` 요청은 단순하지만 Emotet 초기 캠페인에서도 유사 구조 사용됨
- `Entropy` 수치나 특정 단어 조합이 **Emotet 전파와 유사한 텍스트 패턴**을 만들어냈을 가능성

---

#### 샘플 2 – `pid=1770202`
- **예측 확률:** 53.5%
- **실제:** Benign
- **예측:** Banking_Emotet_Family

**요약 내용:**
```
Detected Protocol: HTTP(S)
Entropy: 5.59
Signature: 474554202f204854
Host: betheft.com
User-Agent: Mozilla/5.0 (Linux; Android 8.0.0; ...)
```

**해석:**
- `.com` 도메인이지만 `betheft.com`이라는 명칭이 **금융 탈취(banking theft)** 관련 키워드로 오인될 수 있음
- 모바일 User-Agent → 실제 Emotet도 모바일 브라우저 UA를 가장함
- `betheft`, `Host`, `User-Agent`, `GET` 조합이 Emotet 특징과 겹쳤을 가능성

---

#### 샘플 3 – `pid=1610675`
- **예측 확률:** 89.4%
- **실제:** Benign
- **예측:** Banking_Emotet_Family

**요약 내용:**
```
Detected Protocol: HTTP(S)
Entropy: 7.96 (높음)
Signature: 474554202f204854
Length: 1404 bytes

GET / HTTP/1.1
Host: nationcommunitysurvey.com
Accept-Language: en-US
```

**해석:**
- `Entropy` 값이 높아 압축 또는 난독화된 콘텐츠 포함 가능성 → Emotet 특징 중 하나
- 도메인 이름 `nationcommunitysurvey.com` → 피싱 혹은 설문조사 위장 패턴으로 오해 가능
- 일반적인 요청 같지만, Emotet에서 종종 이런 스타일의 도메인을 이용

---

### 요약

| pid | 실제 | 예측 | 주요 오탐 원인 |
|-----|------|------|----------------|
| 1572623 | Benign | Emotet | 의심 도메인 + 일반적 요청 패턴 |
| 1770202 | Benign | Emotet | 도메인 이름(`betheft`) + UA 조합 |
| 1610675 | Benign | Emotet | 높은 엔트로피 + 설문 위장 도메인 |

---

이처럼 Benign 요청도 **표현, 단어, 도메인 이름, 구조**에 따라  
악성 트래픽으로 오인될 수 있으며, 이를 통해 **모델 민감도 조정이나 예외 처리 설계**가 필요함을 알 수 있습니다.


### 미탐 샘플 해설 (실제: 악성, 예측: Benign)

---

#### 샘플 1 – `pid=1288841`
- **예측 확률:** 80.9%
- **실제:** Recon_C2
- **예측:** Benign

**요약 내용:**
```
Detected Protocol: Unknown (Possibly Encrypted or Custom Protocol)
Entropy: 7.63 (높음)
Length: 727
Hex Signature: 1603030080...
```

**해석:**
- TLS Handshake 시작 패턴 (`16 03 03`) → 암호화 통신 초입부
- Recon/C2 활동 중 `Beaconing` 트래픽과 유사할 수 있음
- 하지만 암호화된 내용이라 **모델이 읽을 수 있는 단서 부족** → Benign으로 오해

---

#### 샘플 2 – `pid=396972`
- **예측 확률:** 82.8%
- **실제:** Exploit_Kit
- **예측:** Benign

**요약 내용:**
```
Detected Protocol: Unknown (Possibly Custom Protocol)
Entropy: 7.71
Length: 176
```

**해석:**
- `Exploit Kit`은 흔히 iframe, JS 인젝션 등으로 작동 → 이 샘플은 아주 짧은 트래픽
- 모델이 Exploit의 특정 키워드나 명령어를 **감지하지 못했을 가능성**
- 패턴이 너무 희미하거나 짧아서 정상 트래픽으로 오해

---

#### 샘플 3 – `pid=766866`
- **예측 확률:** 81.3%
- **실제:** Recon_C2
- **예측:** Benign

**요약 내용:**
```
Detected Protocol: Unknown (Possibly Encrypted)
Entropy: 6.43
Length: 757
```

**해석:**
- 마찬가지로 암호화된 데이터이거나 패턴이 일반적인 TLS 트래픽처럼 보였을 수 있음
- Recon 트래픽 특성상 **주기적인 Beaconing이나 C2 지시문**이 있지만, 해당 샘플에서는 그 패턴이 약했을 수 있음
- 결과적으로, Benign과 구분이 모호한 구조

---

### 요약

| pid | 실제 | 예측 | 주요 미탐 원인 |
|-----|------|------|----------------|
| 1288841 | Recon_C2 | Benign | TLS 암호화로 의미 파악 어려움 |
| 396972  | Exploit_Kit | Benign | 단편적 or 희미한 악성 코드 |
| 766866  | Recon_C2 | Benign | 반복성/Beacon 패턴 불충분 |

---

이러한 미탐(FN)은 실제 보안에서 가장 치명적인 유형입니다.  
모델이 악성을 놓치지 않게 하려면 **암호화된 영역에 대한 간접 특징 학습**, **Beacon 패턴 모델링**,  
또는 **멀티모달(헤더+페이로드+메타 정보)** 학습이 중요합니다.
